# 1. Autograd
## 1.1 梯度

autograd类的原理其实就是一个雅克比矩阵向量积计算引擎.   
在张量间的计算过程中，如果在所有输入中，有一个输入需要求导，那么输出一定会需要求导；相反，只有当所有输入都不需要求导的时候，输出才会不需要。

In [1]:
import torch

In [2]:
x = torch.ones([2, 2], requires_grad=False)    # 输入
w1 = torch.tensor(2.0, requires_grad=True)     # 权重
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

# forward()
l1 = x * w1
l2 = l1 + w2
# l2.retain_grad() 保留非叶子节点的梯度
# l1.register_hook(lambda grad: print('l1 grad: ', grad)) 只打印导数信息，不保存
# 思考： pytorch中的钩子（Hook）有何作用？
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()


print(w1.data, w1.grad, w1.grad_fn)


print(l1.data, l1.grad, l1.grad_fn)


print(loss.data, loss.grad, loss.grad_fn)

tensor(2.) None None
tensor([[2., 2.],
        [2., 2.]]) None <MulBackward0 object at 0x000001FA91236888>
tensor(40.) None <MeanBackward0 object at 0x000001FA95C020C8>


c:\Users\16383\anaconda3\envs\pytorch\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:482.)
  return self._grad


正在访问 非叶子张量的 .grad 属性。 在 autograd.backward() 期间不会填充其 .grad 属性。    
如果您确实希望为非叶子张量的 .grad 字段，请在非叶子张量上使用 .retain_grad() 。

In [13]:
loss.backward()

print(w1.grad, w2.grad, w3.grad)

print(l1.grad, l2.grad, l3.grad, l4.grad, loss.grad)

tensor(28.) tensor(8.) tensor(10.)
None None None None None


## 1.2 叶子张量

在调用backward()时,只有当requires_grad和is_leaf同时为真时,才会计算节点的梯度值.

In [14]:
a = torch.ones([2, 2], requires_grad=True)
print(a.is_leaf)

b = a + 2
print(b.is_leaf)

True
False


对于 requires_grad=False 的 tensor 来说，我们约定俗成地把它们归为叶子张量。

当 requires_grad=True 的时候，如何判断是否是叶子张量：当这个 tensor 是用户创建的时候，它是一个叶子节点，当这个 tensor 是由其他运算操作产生的时候，它就不是一个叶子节点。

思考：为什么需要叶子节点？     
那些非叶子结点，是通过用户所定义的叶子节点的一系列运算生成的，也就是这些非叶子节点都是中间变量，一般情况下，用户不会去使用这些中间变量的导数，所以为了节省内存，它们在用完之后就被释放了。

## 1.3 in-place

inplace 指的是在不更改变量的内存地址的情况下，直接修改变量的值。


In [20]:
# 情景 1
a = torch.tensor([3.0, 1.0])
print(id(a)) 
print(a._version)
print('-----')
a = a.exp()
print(id(a))
print(a._version)


print('*****')


# 情景 2
a = torch.tensor([3.0, 5.0, 7.0])
print(id(a)) 
print(a._version)
print('-----')
a[0] = 10
print(id(a)) 
print(a._version)

2336500244368
0
-----
2336500417376
0
*****
2337898266432
0
-----
2337898266432
1


In [21]:
# 错误举例：发生in-place操作
a = torch.tensor([1.0, 3.0], requires_grad=True)
b = a + 2
print(b._version) # 0

loss = (b * b).mean()
b[0] = 1000.0
print(b._version) # 1

loss.backward()
# 在正向传播过程中，求导系统记录的 b 的 version 是0，
# 但是在进行反向传播的过程中，求导系统发现 b 的 version 变成1了，所以就会报错了。

0
1


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [2]], which is output 0 of struct torch::autograd::CopySlices, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [22]:
# 错误举例：叶子节点变成了非叶子节点
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf)

a[:] = 0
print(a, a.is_leaf)

loss = (a*a).mean()
loss.backward()

tensor([10.,  5.,  2.,  3.], requires_grad=True) True


RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

In [12]:
# 正确实例
# 方法一
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf, id(a))
loss = (a*a).mean()
loss.backward()
print(a.grad)

# a.data.fill_(10.)
a.detach().fill_(10.)
print(a, a.is_leaf, id(a))


loss = (a*a).mean()
loss.backward()
print(a.grad)
print(a.grad_fn)
print(a.requires_grad)

tensor([10.,  5.,  2.,  3.], requires_grad=True) True 2175670744712
tensor([5.0000, 2.5000, 1.0000, 1.5000])
tensor([10., 10., 10., 10.], requires_grad=True) True 2175670744712
tensor([10.0000,  7.5000,  6.0000,  6.5000])
None
True


tensor.detach() 返回一个新的tensor，从当前计算图中分离下来。但是仍指向原变量的存放位置，不同之处只是requirse_grad为false。新得到的这个tensor永远不需要计算梯度，不具有grad。

即使之后重新将它的requires_grad置为true,它也不会具有梯度grad。这样我们就会继续使用这个新的tensor进行计算，后面当我们进行反向传播时，到该调用detach()的tensor就会停止，不能再继续向前进行传播。

注意：使用detach返回的tensor和原始的tensor共同一个内存，即一个修改另一个也会跟着改变。

In [24]:
# 方法二
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf)


with torch.no_grad():
    a[:] = 10.
print(a, a.is_leaf)


loss = (a*a).mean()
loss.backward()
print(a.grad)

tensor([10.,  5.,  2.,  3.], requires_grad=True) True
tensor([10., 10., 10., 10.], requires_grad=True) True
tensor([5., 5., 5., 5.])
